## Motion Correction using the NoRMCorre algorithm

Can use both rigid and piecewise motion correction. The original motion correction algorithm is described in this paper:
https://www.sciencedirect.com/science/article/pii/S0165027017302753?via%3Dihub

More details and tips here: https://caiman.readthedocs.io/en/master/CaImAn_Tips.html#motion-correction-tips

And an example pipeline here:  https://github.com/flatironinstitute/CaImAn/blob/6c5c3b6117b71b6e8b44f62fc26fd3b3d914de12/demos/notebooks/demo_motion_correction.ipynb

In [1]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import numpy as np
import os
import pathlib

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params


import logging
from fancylog import fancylog
import fancylog as package


try:
    cv2.setNumThreads(0)
except:
    pass

from fcutils.file_io.io import load_yaml, save_yaml
from fcutils.file_io.utils import check_create_folder, get_file_name

from movie_visualizer import compare_videos
from utils import start_server, load_params, add_to_params_dict

c, dview, n_processes = start_server()

from IPython.display import clear_output
%config InlineBackend.figure_format = "retina"

## Get file paths
Get the path of the video to motion correct. Need to have done preprocessing first and have specified in `analysis_metadata.yml` which video to use for motion correction.

In [2]:
# Get files to process
fld = 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05' # <- which folder/recording we are working on
metadata = load_yaml(os.path.join(fld, "01_PARAMS", "analysis_metadata.yml"))


if not 'video_for_mc' in metadata.keys() or metadata['video_for_mc'] is None:
    raise ValueError("Do preprocessing first")

vid_to_correct = os.path.join(fld, metadata[metadata['video_for_mc']])
if not os.path.isfile(vid_to_correct):
    raise ValueError(f"The file you want to use for correction doesn't exist: {vid_to_correct}")

fnames    = [vid_to_correct]  # video to processprocessed
base_name = pathlib.Path(fnames[0]).stem # used to save mmepped data

output_fld = os.path.join(fld, metadata['outputfld']) # plots and other stuff will be saved here
check_create_folder(output_fld)

videos = {'raw': os.path.join(metadata['fld'], metadata['ffcsub_video'])} # use this to store paths to videos generated during analysis

## Set up MC params

Each session's parameters are saved in fld > 01_PARAMS > params.yml

In [3]:
# dataset dependent parameters
frate = 10.                       # movie frame rate
decay_time = 2.              # length of a typical transient in seconds


# Load recording specific params and fill them up
prms = load_params(fld)['motion_correction']
prms = add_to_params_dict(prms, fnames=fnames, fr=frate, decay_time=decay_time)


opts = params.CNMFParams(params_dict=prms)


# Setup logging
logging_file = fancylog.start_logging(os.path.join(fld, "02_LOGS"), package, file_log_level="INFO", variables=[opts], verbose=False, filename='motion_correction_logs')

with open( os.path.join(output_fld, "log_file_path.txt"), "w+") as t: # save in the output folder path to log file
    t.write(logging_file)

# Write down metadata about analysis
logging.info("ANALYSIS METADATA FILE:")
for k,v in metadata.items():
    logging.info(f"{k}:  {v}")
logging.info(f"Output folder: {output_fld}")

INFO:root:Starting logging
2020-04-18 17:03:39 PM - INFO - MainProcess fancylog.py:271 - Starting logging
INFO:root:Multiprocessing-logging module not found, not logging multiple processes.
2020-04-18 17:03:39 PM - INFO - MainProcess fancylog.py:273 - Multiprocessing-logging module not found, not logging multiple processes.
INFO:root:ANALYSIS METADATA FILE:
2020-04-18 17:03:39 PM - INFO - MainProcess <ipython-input-3-f4128f1d81a6>:21 - ANALYSIS METADATA FILE:
INFO:root:experimenter:  Federico
2020-04-18 17:03:39 PM - INFO - MainProcess <ipython-input-3-f4128f1d81a6>:23 - experimenter:  Federico
INFO:root:fld:  D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05
2020-04-18 17:03:39 PM - INFO - MainProcess <ipython-input-3-f4128f1d81a6>:23 - fld:  D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05
INFO:root:outputfld:  MC_output_crop_downscale
2020-04-18 17:03:39 PM - INFO - MainProcess <ipython-input-3-f4128f1d81a6>:23 - outputfld:  MC_output_crop_do

# Motion Correction
The background signal in micro-endoscopic data is very strong and makes the motion correction challenging. 
As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the background and enhance spatial landmarks. 
The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data).
After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data so no information is lost.


After the motion correction shifts are computed on the transformed and normalized videos, they are used to motion correct the raw video. This is so that the raw, motion corrected, video can be used for quality evaluation and for CNMF-E fitting. Generally motion correction will be computed on a transformed variant of the raw video (e.g. fft bandpass filtered or normalized), but for signal extraction or evaluating the quality of the MC one might want to look at the raw video motion corrected. To do that, the shifts computed on the transformed video will be applied to the raw video, so that that too is motion corrected


## Rigid motion correction
### Compute shifts

In [4]:
logging.info(f"Starting rigid mc: {prms['niter_rig']} iterations.")

mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

# correct for rigid motion correction and save the file (in memory mapped form)
_ = mc.motion_correct(save_movie=True)

# Now save in C order for CNMF-E 
bord_px = 0
bord_px = 0 if prms['border_nan'] is 'copy' else bord_px
motioncorrected = cm.mmapping.save_memmap(mc.mmap_file, base_name=base_name+"_rig_", order='C', border_to_0=bord_px)
videos['transf_rig_mc'] = motioncorrected

logging.info(f"Rigid motion corrected video was saved at:\n {motioncorrected}")


INFO:root:Starting rigid mc: 1 iterations.
2020-04-18 17:03:40 PM - INFO - MainProcess <ipython-input-4-babe6ea3cd2e>:1 - Starting rigid mc: 1 iterations.
INFO:root:Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft_rig__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap
2020-04-18 17:03:40 PM - INFO - MainProcess motion_correction.py:3017 - Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft_rig__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap
INFO:root:** Starting parallel motion correction **
2020-04-18 17:03:40 PM - INFO - MainProcess motion_correction.py:3030 - ** Starting parallel motion correction **
INFO:root:** Finished parallel motion correction **
2020-04-18 17:04:58 PM - INFO - MainProcess motion_correction.py:3038 - ** Finished parallel motion correction **
INFO:root:Rigid motion corrected video was saved at:
 D:\Dropbox (UCL -

### Apply shifts to raw video

In [5]:
apply_to = os.path.join(metadata['fld'], metadata['ffcsub_video'])
apply_to_base_name = os.path.join(fld, pathlib.Path(apply_to).stem+"_rig") # used to save mmepped data

logging.info(f"Applying shifts to ffcsub (not normalized) video: {apply_to}")

raw_mc_filepath_F = mc.apply_shifts_movie(apply_to, save_memmap=True, save_base_name=apply_to_base_name, order="F", remove_min=True)
logging.info(f"Saved motion corrected raw video as: {raw_mc_filepath_F}\n")

# Now save in C order
raw_motioncorrected = cm.mmapping.save_memmap([raw_mc_filepath_F], base_name=apply_to_base_name, order='C', border_to_0=bord_px)
videos['raw_rig_mc'] = raw_motioncorrected
logging.info(f"Saved motion corrected raw video (C order) as: {raw_motioncorrected}\n")

INFO:root:Applying shifts to ffcsub (not normalized) video: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff
2020-04-18 17:05:07 PM - INFO - MainProcess <ipython-input-5-888702039146>:4 - Applying shifts to ffcsub (not normalized) video: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff
INFO:root:Saved motion corrected raw video as: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig_d1_109_d2_92_d3_1_order_F_frames_22662_.mmap

2020-04-18 17:05:25 PM - INFO - MainProcess <ipython-input-5-888702039146>:7 - Saved motion corrected raw video as: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig_d1_109_d2_92_d3_1_order_F_frames_22662_.mmap

INFO:root:Saved motion corrected raw video (C order) as: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric

## Piecewise rigid motion correction
### Compute shifts

In [6]:
# motion correct piecewise rigid
mc.pw_rigid = True  # turn the flag to True for pw-rigid motion correction
mc.template = mc.mmap_file  # use the results of the rigid motion corrction to save in computation
_ = mc.motion_correct(save_movie=True, template=mc.total_template_rig)

# Now save in C order for CNMF-E 
bord_px = 0 if prms['border_nan'] is 'copy' else bord_px
pw_motioncorrected = cm.mmapping.save_memmap(mc.mmap_file, base_name=base_name+"_els_", order='C', border_to_0=bord_px)
videos['transf_pw_mc'] = pw_motioncorrected

logging.info(f"Piecewise motion corrected video was saved at:\n {pw_motioncorrected}")



INFO:root:Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft_els__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap
2020-04-18 17:05:35 PM - INFO - MainProcess motion_correction.py:3017 - Saving file as D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft_els__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap
INFO:root:** Starting parallel motion correction **
2020-04-18 17:05:35 PM - INFO - MainProcess motion_correction.py:3030 - ** Starting parallel motion correction **
INFO:root:** Finished parallel motion correction **
2020-04-18 17:06:43 PM - INFO - MainProcess motion_correction.py:3038 - ** Finished parallel motion correction **
INFO:root:Piecewise motion corrected video was saved at:
 D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft_els__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap
202

### Apply shifts to raw video

In [7]:
apply_to = os.path.join(metadata['fld'], metadata['ffcsub_video'])
apply_to_base_name = os.path.join(fld, pathlib.Path(apply_to).stem+"_els") # used to save mmepped data

logging.info(f"Applying shifts to ffcsub (not normalized) video: {apply_to}")

raw_mc_els_filepath_F = mc.apply_shifts_movie(apply_to, save_memmap=True, save_base_name=apply_to_base_name, order="F", remove_min=True)
logging.info(f"Saved motion corrected raw video as: {raw_mc_els_filepath_F}\n")

# Now save in C order
raw_pw_motioncorrected = cm.mmapping.save_memmap([raw_mc_els_filepath_F], base_name=apply_to_base_name, order='C', border_to_0=bord_px)
videos['raw_pw_mc'] = raw_pw_motioncorrected
logging.info(f"Saved motion corrected raw video (C order) as: {raw_pw_motioncorrected}\n")

INFO:root:Applying shifts to ffcsub (not normalized) video: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff
2020-04-18 17:06:53 PM - INFO - MainProcess <ipython-input-7-510b9cbc6680>:4 - Applying shifts to ffcsub (not normalized) video: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff
INFO:root:Saved motion corrected raw video as: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_els_d1_109_d2_92_d3_1_order_F_frames_22662_.mmap

2020-04-18 17:07:04 PM - INFO - MainProcess <ipython-input-7-510b9cbc6680>:7 - Saved motion corrected raw video as: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_els_d1_109_d2_92_d3_1_order_F_frames_22662_.mmap

INFO:root:Saved motion corrected raw video (C order) as: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric

## End
Save a couple things that will be useful for the next few steps and you're done. 

In [8]:
savepath = os.path.join(output_fld, "video_paths.yml")
save_yaml(savepath, videos)

logging.info(f"VIDEOS: {videos}")
logging.info(f"Saving video paths at {savepath}")


INFO:root:VIDEOS: {'raw': 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05\\19JUN05_BF164p1_v1_ds126_crop_ffcSub.tiff', 'transf_rig_mc': 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05\\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft_rig__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap', 'raw_rig_mc': 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05\\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig_d1_109_d2_92_d3_1_order_C_frames_22662_.mmap', 'transf_pw_mc': 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05\\19JUN05_BF164p1_v1_ds126_crop_ffcSub_div_fft_els__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap', 'raw_pw_mc': 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05\\19JUN05_BF164p1_v1_ds126_crop_ffcSub_els_d1_109_d2_92_d3_1_order_C_frames_22662_.mmap'}
2020-04-18 17:07:14 PM - INFO - MainProcess <ipython-input-8-d25d3c3b3f9a>:4 - VIDEOS: {'raw': 'D:\\Dropbox (U